In [1]:
import sys
from PyQt5.QtWidgets import *
import win32com.client
import ctypes
 
################################################
# PLUS 공통 OBJECT
g_objCodeMgr = win32com.client.Dispatch('CpUtil.CpCodeMgr')
g_objCpStatus = win32com.client.Dispatch('CpUtil.CpCybos')
g_objCpTrade = win32com.client.Dispatch('CpTrade.CpTdUtil')
 
 
################################################
# PLUS 실행 기본 체크 함수
def InitPlusCheck():
    # 프로세스가 관리자 권한으로 실행 여부
    if ctypes.windll.shell32.IsUserAnAdmin():
        print('정상: 관리자권한으로 실행된 프로세스입니다.')
    else:
        print('오류: 일반권한으로 실행됨. 관리자 권한으로 실행해 주세요')
        return False
 
    # 연결 여부 체크
    if (g_objCpStatus.IsConnect == 0):
        print("PLUS가 정상적으로 연결되지 않음. ")
        return False
 
    # # 주문 관련 초기화 - 계좌 관련 코드가 있을 때만 사용
    # if (g_objCpTrade.TradeInit(0) != 0):
    #     print("주문 초기화 실패")
    #     return False
 
    return True
 

In [4]:
 
class CpMarketEye:
    def __init__(self):
        self.objRq = win32com.client.Dispatch("CpSysDib.MarketEye")
        self.RpFiledIndex = 0
 
 
    def Request(self, codes, dataInfo):
        # 0: 종목코드 4: 현재가 20: 상장주식수 67: PER 71:자본금 72: 액면가
        rqField = [0, 4, 20, 67, 71, 72]  # 요청 필드
 
        self.objRq.SetInputValue(0, rqField)  # 요청 필드
        self.objRq.SetInputValue(1, codes)  # 종목코드 or 종목코드 리스트
        self.objRq.BlockRequest()
 
        # 현재가 통신 및 통신 에러 처리
        rqStatus = self.objRq.GetDibStatus()
        print("통신상태", rqStatus, self.objRq.GetDibMsg1())
        if rqStatus != 0:
            return False
 
        cnt = self.objRq.GetHeaderValue(2)
 
        for i in range(cnt):
            code = self.objRq.GetDataValue(0, i)  # 코드
            cur = self.objRq.GetDataValue(1, i)  # 종가
            listedStock = self.objRq.GetDataValue(2, i)  # 상장주식수
            per = self.objRq.GetDataValue(3, i)  # per
            base = self.objRq.GetDataValue(4, i)  # base
            face = self.objRq.GetDataValue(5, i)  # 액면가
 
            maketAmt = listedStock * cur
            if g_objCodeMgr.IsBigListingStock(code) :
                maketAmt *= 1000
                listedStock *= 1000
                print(code, maketAmt)
 
            # key(종목코드) = tuple(상장주식수, 시가총액)
            dataInfo[code] = (listedStock, maketAmt, per, base, face)
 
        return True
 
class CMarketTotal():
    def __init__(self):
        self.dataInfo = {}
 
 
    def GetAllMarketTotal(self):
        codeList = g_objCodeMgr.GetStockListByMarket(1)  # 거래소
        codeList2 = g_objCodeMgr.GetStockListByMarket(2)  # 코스닥
        allcodelist = codeList + codeList2
        allcodelist = ("U001",) + codeList + codeList2
        print('전 종목 코드 %d, 거래소 %d, 코스닥 %d' % (len(allcodelist), len(codeList), len(codeList2)))
 
        objMarket = CpMarketEye()
        rqCodeList = []
        for i, code in enumerate(allcodelist):
            rqCodeList.append(code)
            if len(rqCodeList) == 200:
                objMarket.Request(rqCodeList, self.dataInfo)
                rqCodeList = []
                continue
        # end of for
 
        if len(rqCodeList) > 0:
            objMarket.Request(rqCodeList, self.dataInfo)
 
    def PrintMarketTotal(self):
 
        # 시가총액 순으로 소팅
        data2 = sorted(self.dataInfo.items(), key=lambda x: x[1][1], reverse=True)
 
        print('전종목 시가총액 순 조회 (%d 종목)' % (len(data2)))
        for item in data2:
            name = g_objCodeMgr.CodeToName(item[0])
            listed = item[1][0]
            markettot = item[1][1]
            print('%s 상장주식수: %s, 시가총액 %s' %(name, format(listed, ','), format(markettot, ',')))
 

In [5]:
objMarketTotal = CMarketTotal()
objMarketTotal.GetAllMarketTotal()
objMarketTotal.PrintMarketTotal()

전 종목 코드 2973, 거래소 1558, 코스닥 1414
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
A005930 285654068700000
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
전종목 시가총액 순 조회 (2973 종목)
삼성전자 상장주식수: 5,969,782,000, 시가총액 285,654,068,700,000
SK하이닉스 상장주식수: 728,002,365, 시가총액 59,623,393,693,500
삼성바이오로직스 상장주식수: 66,165,000, 시가총액 40,029,825,000,000
NAVER 상장주식수: 164,263,395, 시가총액 34,988,103,135,000
삼성전자우 상장주식수: 822,886,700, 시가총액 33,697,210,365,000
셀트리온 상장주식수: 134,706,932, 시가총액 29,298,757,710,000
LG화학 상장주식수: 70,592,343, 시가총액 24,460,246,849,500
LG생활건강 상장주식수

인포바인 상장주식수: 3,192,883, 시가총액 62,101,574,350
코닉글로리 상장주식수: 50,642,595, 시가총액 62,037,178,875
바른전자 상장주식수: 51,256,571, 시가총액 62,020,450,910
인포뱅크 상장주식수: 8,693,978, 시가총액 61,901,123,360
STX중공업 상장주식수: 28,365,976, 시가총액 61,837,827,680
티플랙스 상장주식수: 24,268,402, 시가총액 61,763,083,090
네온테크 상장주식수: 35,695,160, 시가총액 61,752,626,800
디케이락 상장주식수: 8,449,912, 시가총액 61,599,858,480
신한 레버리지 은 선물 ETN(H) 상장주식수: 10,000,000, 시가총액 61,500,000,000
라이브파이낸셜 상장주식수: 39,116,090, 시가총액 61,412,261,300
대림B&Co 상장주식수: 16,672,240, 시가총액 61,353,843,200
우진비앤지 상장주식수: 25,967,013, 시가총액 61,282,150,680
롯데지주우 상장주식수: 987,623, 시가총액 61,232,626,000
한국팩키지 상장주식수: 25,000,000, 시가총액 61,125,000,000
아이즈비전 상장주식수: 15,772,574, 시가총액 61,118,724,250
서울식품 상장주식수: 337,410,000, 시가총액 61,071,210,000
W홀딩컴퍼니 상장주식수: 219,025,237, 시가총액 60,889,015,886
TRUE 레버리지 나스닥 100 ETN 상장주식수: 5,000,000, 시가총액 60,825,000,000
서암기계공업 상장주식수: 12,600,000, 시가총액 60,795,000,000
그린플러스 상장주식수: 4,823,871, 시가총액 60,780,774,600
린드먼아시아 상장주식수: 13,500,000, 시가총액 60,615,000,000
장원테크 상장주식수: 33,862,106, 시가총액 60

In [6]:
data2 = sorted(objMarketTotal.dataInfo.items(), key=lambda x: x[1][1], reverse=True)

In [7]:
data2[0]

('A005930', (5969782000, 285654068700000, 15.109999656677246, 778046, 100))

In [8]:
MarketTotal = []

for item in data2:
    name = g_objCodeMgr.CodeToName(item[0])
    listed = item[1][0]
    markettot = item[1][1]
    per = item[1][2]
    eps = item[1][3]
    face = item[1][4]
    
    MarketTotal.append([name, item[0], listed, markettot, per, eps, face])

In [9]:
import pandas as pd

In [10]:
format(1e+10, ",")

'10,000,000,000.0'

In [11]:
format(1e+12, ",")

'1,000,000,000,000.0'

In [12]:
pdMarketTotal = pd.DataFrame(MarketTotal)

In [13]:
len(pdMarketTotal)

2973

In [14]:
pdMarketTotal.to_csv("../data/KRX_LIST.csv", index=None)

In [15]:
pdMarketTotal.columns = "Name,Code,Stock,capitalization,per,base,face".split(",")

In [16]:
pdMarketTotal.loc[pdMarketTotal.Code == "U001"]

,Name,Code,Stock,capitalization,per,base,face
2972,종합주가지수,U001,0,0.0,0.0,0,0


In [17]:
pdMarketTotal.loc[pdMarketTotal.Name == "KODEX 200선물인버스2X"]

,Name,Code,Stock,capitalization,per,base,face
96,KODEX 200선물인버스2X,A252670,323800000,2.188888e+12,0.0,0,0


In [18]:
pdMarketTotal.loc[pdMarketTotal.Name == "KODEX 미국S&P500선물(H)"]

,Name,Code,Stock,capitalization,per,base,face
1483,KODEX 미국S&P500선물(H),A219480,5750000,7.820000e+10,0.0,0,0


In [19]:
pdMarketTotal.loc[pdMarketTotal.Name == "KODEX 미국S&P500선물(H)"]

,Name,Code,Stock,capitalization,per,base,face
1483,KODEX 미국S&P500선물(H),A219480,5750000,7.820000e+10,0.0,0,0


In [20]:
pdMarketTotal.loc[pdMarketTotal.Code == "A950140"]

,Name,Code,Stock,capitalization,per,base,face
1346,잉글우드랩,A950140,19867866,9.178954e+10,40.52,0,0


In [21]:
pdMarketTotal.loc[pdMarketTotal.Code == "A900140"]

,Name,Code,Stock,capitalization,per,base,face
656,엘브이엠씨홀딩스,A900140,51120439,2.341316e+11,0.0,12,0


In [22]:
pdMarketTotal.loc[pdMarketTotal.Code == "A005930"]

,Name,Code,Stock,capitalization,per,base,face
0,삼성전자,A005930,5969782000,2.856541e+14,15.11,778046,100


In [23]:
pdMarketTotal.loc[pdMarketTotal.Code == "A000660"]

,Name,Code,Stock,capitalization,per,base,face
1,SK하이닉스,A000660,728002365,5.962339e+13,29.6,3657652,5000


In [24]:
# pdMarketTotal_not_etf = pdMarketTotal.loc[(
#     (pdMarketTotal["Name"].apply(lambda x: x.find("KODEX") < 0)) &
#     (pdMarketTotal["Name"].apply(lambda x: x.find("TIGER") < 0)) &
#     (pdMarketTotal["Name"].apply(lambda x: x.find("KBSTAR") < 0)) &
#     (pdMarketTotal["Name"].apply(lambda x: x.find("HANARO") < 0)) &
#     (pdMarketTotal["Name"].apply(lambda x: x.find("ARIRANG") < 0)) &
#     (pdMarketTotal["Name"].apply(lambda x: x.find("KINDEX") < 0)) &
#     (pdMarketTotal["Name"].apply(lambda x: x.find("KOSEF") < 0)) &
#     (pdMarketTotal["Name"].apply(lambda x: x.find("SMART") < 0)) &
#     (pdMarketTotal["Name"].apply(lambda x: x.find("TREX") < 0)) &
#     (pdMarketTotal["Name"].apply(lambda x: x.find("OTM") < 0)) &
#     (pdMarketTotal["Name"].apply(lambda x: x.find("ATM") < 0)) &
#     (pdMarketTotal["Name"].apply(lambda x: x.find("ETN") < 0)) &
#     (pdMarketTotal["Name"].apply(lambda x: x.find("ETF") < 0)) 
# )]

In [25]:
pdMarketTotal = pdMarketTotal[pdMarketTotal.Code.apply(lambda x: not x.endswith("K"))]
pdMarketTotal = pdMarketTotal[pdMarketTotal.Code.apply(lambda x: not x.startswith("Q"))]

In [30]:
KOSPI = pdMarketTotal.loc[(
    (pdMarketTotal.Stock == 0)
), "Name,Code,Stock,capitalization".split(",")]

In [32]:
KOSPI["type"] = "index"

In [33]:
STOCK = pdMarketTotal.loc[(
    ((pdMarketTotal.face > 0) & 
     (pdMarketTotal.base > 0) & 
     (pdMarketTotal.per > 0) & 
     (pdMarketTotal.Stock > 1e+7) &
     (pdMarketTotal.capitalization > 1e+12))
), "Name,Code,Stock,capitalization".split(",")]

In [35]:
STOCK["type"] = "stock"

In [36]:
ETF = pdMarketTotal.loc[(
    ((pdMarketTotal.face == 0) & 
     (pdMarketTotal.base == 0) & 
     (pdMarketTotal.per == 0) & 
     (pdMarketTotal.capitalization > 1e+10))
), "Name,Code,Stock,capitalization".split(",")]

In [37]:
ETF["type"] = 'etf'

In [38]:
concat = pd.concat([KOSPI, STOCK, ETF])

In [39]:
concat

,Name,Code,Stock,capitalization,type
2972,종합주가지수,U001,0,0.000000e+00,index
0,삼성전자,A005930,5969782000,2.856541e+14,stock
1,SK하이닉스,A000660,728002365,5.962339e+13,stock
2,삼성바이오로직스,A207940,66165000,4.002982e+13,stock
3,NAVER,A035420,164263395,3.498810e+13,stock
...,...,...,...,...,...
2615,KINDEX 싱가포르리츠,A316300,1000000,1.020000e+10,etf
2617,TIGER 200 중공업,A139230,5320000,1.018780e+10,etf
2618,KBSTAR 고배당,A266160,1300000,1.017250e+10,etf
2619,KODEX TRF5050,A329660,1000000,1.015000e+10,etf


In [40]:
concat.to_csv("../data/SELECT_LIST.csv", index=None)